# Introduction:

In these series of notebook we expanded the work with our feature space. We reduced smartly the number of features. We  also analyzed variables with regard the predictor we want to classify. Some ideas to ponder and you can think about what we can do with our variables and what are the effect of this actions:

+ Remove features with little variance. (imagine an extreme a zero variance feature, don't teach us anything!)
+ Univariate feauture selection. (There are a lot here, but I want to explore mutual information and corrolation.)
+ Recursive Feature elimination. (I will not use this one, I prefer creating a simple model and see different variables.)
+ Model with L1 regularization. (this model if it finds a variable with little predicting power it will just make it zero.)

My first ever repo have more about EDA and explaining what is behind the data. But this repo my goal is to create useful analysis that can be easy to replicate on your notebook, and taking a whole bunch of lines of code + text make it harder to look where the nice rounded functions are exactly.

In [30]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
# new library, we haven't used this one before in this repo
from sklearn.preprocessing import TargetEncoder

# remove feature with zero variance by default
from sklearn.feature_selection import VarianceThreshold

from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import r_regression

In [31]:
# Read the data
# This data you can find here: https://www.kaggle.com/c/home-data-for-ml-course/data

X_full = pd.read_csv('train.csv', index_col='Id')

# SalePrice is the target, if there is no target eliminate row associated with it
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X = X_full.copy()
X.drop(['SalePrice'], axis=1, inplace=True)

# Univariate feature selection

In [32]:
# It is always good idea to check the input of a pipeline that only transform and its output!
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [33]:
def simple_pipeline(X,y):

    columns = X.columns
    numerical_col = [col for col in X.columns if str(X[col].dtypes)!='object' ]
    categorical_col = [col for col in X.columns if str(X[col].dtypes)=='object' ]
    
   
    numerical_transformer = Pipeline(steps=[ 
            ("zero_variance_del",VarianceThreshold()),
            ("imputer", KNNImputer(n_neighbors=3))
          ])
    target_transformer =  Pipeline(steps=[
        ('encoder', TargetEncoder(smooth="auto",target_type='continuous')),
        ("zero_variance_del",VarianceThreshold())
    ])

    preprocessor = ColumnTransformer(transformers=
        [("numerical_transformer", numerical_transformer, numerical_col),
         ("target_transformer", target_transformer, categorical_col)
        ])
    
    # Bundle preprocessing and modeling code in a pipeline
    pipe = Pipeline(steps=[('preprocessor', preprocessor)
                         ])
    new_X = pipe.fit_transform(X,y)
    df = pd.DataFrame(new_X, columns=numerical_col+categorical_col)
    return df, pipe

In [34]:
X, pipe = simple_pipeline(X,y)

In [35]:
X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,201814.394580,200589.565875,187476.044540,188287.709377,186712.815097,180693.852124,187984.881910,182539.834852,173170.406344,174676.521359
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,202665.473372,202154.109471,187274.214691,187728.662297,186290.155449,180359.994942,188183.099635,181520.475574,172659.270437,174896.921494
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,200804.625803,201635.761819,186111.111762,186232.805800,184576.320166,178379.834654,185432.676397,179930.938089,171380.182653,172948.626979
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,132478.944769,141274.552634,186111.111762,186232.805800,184576.320166,178379.834654,185432.676397,179930.938089,171380.182653,145356.260017
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,200804.625803,201635.761819,186111.111762,186232.805800,184576.320166,178379.834654,185432.676397,179930.938089,171380.182653,172948.626979


In [36]:
(X.dtypes!="float64")

MSSubClass       False
LotFrontage      False
LotArea          False
OverallQual      False
OverallCond      False
YearBuilt        False
YearRemodAdd     False
MasVnrArea       False
BsmtFinSF1       False
BsmtFinSF2       False
BsmtUnfSF        False
TotalBsmtSF      False
1stFlrSF         False
2ndFlrSF         False
LowQualFinSF     False
GrLivArea        False
BsmtFullBath     False
BsmtHalfBath     False
FullBath         False
HalfBath         False
BedroomAbvGr     False
KitchenAbvGr     False
TotRmsAbvGrd     False
Fireplaces       False
GarageYrBlt      False
GarageCars       False
GarageArea       False
WoodDeckSF       False
OpenPorchSF      False
EnclosedPorch    False
3SsnPorch        False
ScreenPorch      False
PoolArea         False
MiscVal          False
MoSold           False
YrSold           False
MSZoning         False
Street           False
Alley            False
LotShape         False
LandContour      False
Utilities        False
LotConfig        False
LandSlope  

In [37]:
pd.set_option('display.max_rows', None)

In [38]:
def make_mi_scores(X, y):
    mi_scores = mutual_info_regression(X, y, discrete_features='auto')
    r_score = r_regression(X, y)
    # we could also use f_regression to see if the corrolation is significant or not
    # but in this case I just wanted to see amount how those variables move together.
    return pd.DataFrame({"mi_scores":mi_scores,
                        "r_score": r_score}, index=X.columns).sort_values(by='r_score',ascending=False)

mi_scores = make_mi_scores(X, y)
mi_scores.head()  # show a few features with their MI scores

,mi_scores,r_score
OverallQual,0.565285,0.790982
Neighborhood,0.481266,0.725876
GrLivArea,0.482707,0.708624
ExterQual,0.321099,0.688784
BsmtQual,0.323528,0.678324


It looks like there are variables that do not explain much what we want, that is, the price. We always have to keep in mind that these method do not see interaction between variables. This method are very simple and see only "direct" interaction. But let us see what happened if we take out some of these low explainer variables.

In [39]:
# I just invented this metrics but it is a simple idea :) 
mi_scores['overall_explainer'] = 2**mi_scores['mi_scores']+ abs(mi_scores['r_score'])

In [68]:
mi_scores.sort_values(by='overall_explainer',ascending=False).tail()

,mi_scores,r_score,overall_explainer
MiscVal,0.000000,-0.021190,1.021190
Condition2,0.000000,0.019048,1.019048
LandSlope,0.000000,0.017002,1.017002
BsmtFinSF2,0.006323,-0.011378,1.015771
Street,0.000000,-0.009601,1.009601


In [63]:
def selecting_feature(column,threshold):
    features = list(mi_scores[mi_scores[column] >= threshold].index)
    return features

In [64]:
model = RandomForestRegressor(n_estimators=369,random_state=0,n_jobs=-1)
score_all_variables = -1 * cross_val_score(model,X, y,cv=5,scoring="neg_mean_absolute_error")
score_all_variables_mean = score_all_variables.mean()

In [74]:
score_all_variables_mean

17595.434784868397

In [70]:
def selecting_variable_model(column,threshold):
    columns = X.columns
    features = selecting_feature(column,threshold)
    print(f" Variables taking out: {len(columns)-len(features)}")
    print(f" Number of variables now: {len(features)}")
    model = RandomForestRegressor(n_estimators=369,random_state=0,n_jobs=-1)
    scores = -1 * cross_val_score(model, X[features], y,cv=5,scoring="neg_mean_absolute_error")

    print(f" MAE lost for reducing number variables: {(scores.mean())-(score_all_variables_mean)}")
    print(f" MAE now: {(scores.mean())}")

In [71]:
selecting_variable_model("overall_explainer",1.1)

 Variables taking out: 14
 Number of variables now: 65
 MAE lost for reducing number variables: -65.32295170211728
 MAE now: 17530.11183316628


In [72]:
selecting_variable_model("overall_explainer",1.2)

 Variables taking out: 27
 Number of variables now: 52
 MAE lost for reducing number variables: 69.9608790882412
 MAE now: 17665.39566395664


In [73]:
selecting_variable_model("overall_explainer",1.8)

 Variables taking out: 68
 Number of variables now: 11
 MAE lost for reducing number variables: 1562.4989746651627
 MAE now: 19157.93375953356


There is a lot of learning here!!!
Notice that when we take out columns that do not have a lot of predicting power with the price the model improve even though we took out a lot columns. That was unexpected to be honest. I thought it would be a slightly lost in MAE, but it was the other way around. 
The other thing to notice is the model remain a lot of its predicting power even when we take out a lot of its variables.